In [1]:
import warnings
import logging
warnings.filterwarnings("ignore")
warnings.simplefilter('ignore')
import filtros


In [12]:
import copy
import datetime
import glob
import os
import warnings
from functools import lru_cache, partial
from pprint import pprint
from typing import Dict, Tuple

import gymnasium as gym
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import ray
import torch
from IPython.display import HTML, display
from neuralforecast.core import NeuralForecast
from ray.rllib.algorithms.algorithm import Algorithm
from ray.rllib.algorithms.callbacks import DefaultCallbacks
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.env import BaseEnv
from ray.rllib.evaluation import Episode, RolloutWorker
from ray.rllib.policy import Policy
from ray.tune import JupyterNotebookReporter
from ray.tune.registry import register_env
from ray.tune.search import ConcurrencyLimiter
from ray.tune.search.hyperopt import HyperOptSearch
from ray.tune.search.optuna import OptunaSearch
# from ray import train, tune
from tqdm.autonotebook import tqdm

from environments.environments import NeuralForecastingTradingEnv
from utils.utils import build_market_image, prep_forecasts,sharpe_reward

In [13]:
import os

In [14]:

# Load custom CSS file
css = HTML('<link rel="stylesheet" type="text/css" href="./custom.css">') # make sure the path is correct, it fails silently otherwise
display(css)

In [21]:

target_pair='USD_JPY'
symbol='USD_JPY'
time_frame="1h"
checkpoint_path='/Users/sebastiancoombs/git/MultiTrader/Agent_FX'


In [22]:
import warnings
warnings.filterwarnings('ignore')

In [23]:

symbol='USD_JPY'
data=build_market_image(target_pair=symbol,dir='data_forex',time_frame='1h',axis=0)
data=data[data['symbol']==target_pair.replace('/','')].copy()
data['ds']=data.index

data.symbol.unique()

array(['USD_JPY'], dtype=object)

In [24]:
split_date=datetime.datetime(year= 2024, month= 3, day=1)
end_date=split_date+datetime.timedelta(days=14)

split_date=datetime.datetime(year= 2024, month= 3, day=1)
hf_data=data.copy()

hf_train_data=hf_data.groupby('symbol').apply(lambda x: x[:split_date])
hf_test_data=hf_data.groupby('symbol').apply(lambda x: x[split_date:end_date])

hf_train_data=hf_train_data.reset_index(level=0,drop=True).reset_index()
hf_test_data=hf_test_data.reset_index(level=0,drop=True).reset_index()


trade_data=data[data['symbol']==symbol].copy()
train_data=trade_data[:split_date]
test_data=trade_data[split_date:end_date]
# hf_test_data

In [25]:
train_data.head(5)

,open,high,low,close,volume,y,feature_hour_of_day,feature_day_of_week,feature_day_of_month,feature_day_of_year,...,feature_log_volume_15,feature_log_return_20,feature_log_volume_20,feature_log_return_25,feature_log_volume_25,feature_log_return_30,feature_log_volume_30,ds,unique_id,symbol
date_close,,,,,,,,,,,,,,,,,,,,,
2022-01-02 22:00:00,115.118,115.155,115.084,115.125,231,115.125,0.456522,0.5,-0.466667,-0.497260,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-01-02 22:00:00,20,USD_JPY
2022-01-02 23:00:00,115.110,115.158,115.099,115.144,848,115.144,0.500000,0.5,-0.466667,-0.497260,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-01-02 23:00:00,20,USD_JPY
2022-01-03 00:00:00,115.142,115.294,115.124,115.280,1771,115.280,-0.500000,-0.5,-0.433333,-0.494521,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-01-03 00:00:00,20,USD_JPY
2022-01-03 01:00:00,115.279,115.280,115.182,115.213,1401,115.213,-0.456522,-0.5,-0.433333,-0.494521,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-01-03 01:00:00,20,USD_JPY
2022-01-03 02:00:00,115.214,115.279,115.207,115.278,946,115.278,-0.413043,-0.5,-0.433333,-0.494521,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-01-03 02:00:00,20,USD_JPY


In [26]:
model_path='FX_forecasting_model/'

model=NeuralForecast.load(model_path,verbose=False,)

Seed set to 2
Seed set to 16
Seed set to 8
Seed set to 5


In [27]:
os.environ['PYTORCH_ENABLE_MPS_FALLBACK']="1"
train_forecast_array,train_df=prep_forecasts(model=model,df=train_data)
test_forecast_array,test_df=prep_forecasts(model=model,df=test_data)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [28]:
train_env_config=dict(
                name=f'{target_pair}_train',
                forecast_horizon=7,
                context_length=35,
                reward_function=sharpe_reward,
                forecasts=train_forecast_array,
                df = train_df, # Your dataset with your custom features
                positions = [ -.5,-.25,0,.25, .5], # -1 (=SHORT), 0(=OUT), +1 (=LONG)
                trading_fees = 0.001/100, # 0.01% per stock buy / sell (Binance fees)
                borrow_interest_rate= 0.0003/100, # 0.0003% per timestep (one timestep = 1h here)
                max_episode_duration=168,
                verbose=0,
                
                )

test_env_config=copy.deepcopy(train_env_config)
test_env_config['name']=f'{target_pair}_test',

test_env_config['df'] = test_df
test_env_config['forecasts']=test_forecast_array



In [29]:
train_env=NeuralForecastingTradingEnv(**train_env_config)

In [30]:
obs,info=train_env.reset()
obs.shape

(53,)

In [31]:
train_env.df.tail(5)

,open,high,low,close,volume,y,feature_hour_of_day,feature_day_of_week,feature_day_of_month,feature_day_of_year,...,feature_log_volume_20,feature_log_return_25,feature_log_volume_25,feature_log_return_30,feature_log_volume_30,ds,unique_id,symbol,dynamic_feature__0,dynamic_feature__1
date_close,,,,,,,,,,,,,,,,,,,,,
2024-02-29 14:00:00,149.784,149.850,149.478,149.514,10651,149.514,0.108696,0.0,0.433333,-0.338356,...,0.613849,-0.007987,0.045229,-0.008498,0.579577,2024-02-29 14:00:00,20,USD_JPY,0,0
2024-02-29 15:00:00,149.486,149.700,149.208,149.642,10186,149.642,0.152174,0.0,0.433333,-0.338356,...,0.438500,-0.007695,0.054575,-0.007351,0.669092,2024-02-29 15:00:00,20,USD_JPY,0,0
2024-02-29 16:00:00,149.642,149.907,149.636,149.902,10773,149.902,0.195652,0.0,0.433333,-0.338356,...,0.673751,-0.006059,0.341945,-0.005097,0.952971,2024-02-29 16:00:00,20,USD_JPY,0,0
2024-02-29 17:00:00,149.904,150.048,149.892,149.996,6846,149.996,0.239130,0.0,0.433333,-0.338356,...,1.226525,-0.004550,-0.096425,-0.004244,0.562175,2024-02-29 17:00:00,20,USD_JPY,0,0
2024-02-29 18:00:00,149.996,150.019,149.841,149.872,5927,149.872,0.282609,0.0,0.433333,-0.338356,...,1.470564,-0.005510,0.100927,-0.005768,0.309802,2024-02-29 18:00:00,20,USD_JPY,0,0


In [32]:
obs=train_env.reset()
terminated = truncated = False
# env.action_space.sa
while not terminated and not truncated:
    action = train_env.action_space.sample()
    obs, reward, terminated, truncated, info = train_env.step(action)
info['reward']
train_env.results_metrics
info

{'idx': 10480,
 'step': 167,
 'date': numpy.datetime64('2023-09-12T12:00:00.000000000'),
 'position_index': 1,
 'position': -0.25,
 'real_position': -0.25047764249723375,
 'data_volume': 7168,
 'data_high': 147.163,
 'data_y': 147.122,
 'data_symbol': 'USD_JPY',
 'data_low': 146.896,
 'data_unique_id': 20,
 'data_close': 147.122,
 'data_open': 146.897,
 'data_ds': Timestamp('2023-09-12 12:00:00'),
 'portfolio_valuation': 1005.5851332446352,
 'portfolio_distribution_asset': 0,
 'portfolio_distribution_fiat': 1257.4617267500182,
 'portfolio_distribution_borrowed_asset': 1.7120202136857117,
 'portfolio_distribution_borrowed_fiat': 0,
 'portfolio_distribution_interest_asset': 5.136060641057135e-06,
 'portfolio_distribution_interest_fiat': 0.0,
 'reward': 0.09071011913910333}

In [33]:


register_env("train_env", lambda config: NeuralForecastingTradingEnv(**train_env_config))
register_env("test_env", lambda config: NeuralForecastingTradingEnv(**test_env_config))


In [34]:

def trial_str_creator(trial):
    return "{}_{}_trading_agent".format(trial.trainable_name, trial.trial_id)



In [35]:
# type(float(np.float64(1660)))

In [36]:
class TraderCallbacks(DefaultCallbacks):
    
    def on_episode_start(
            self,
            *,
            worker: RolloutWorker,
            base_env: BaseEnv,
            policies: Dict[str, Policy],
            episode: Episode,
            env_index: int,
            **kwargs,
        ):
        # Make sure this episode has just been started (only initial obs
        # logged so far).


        # Create lists to store angles in
        episode.hist_data["portfolio_values"] = []
        episode.hist_data["positions"] = []

    def on_episode_step(
        self,
        *,
        worker: RolloutWorker,
        base_env: BaseEnv,
        policies: Dict[str, Policy],
        episode: Episode,
        env_index: int,
        **kwargs,
    ):
        # Make sure this episode is ongoing.

        portfolio_valuation = episode.last_info_for()['portfolio_valuation']
        position=episode.last_info_for()['position']
        episode.hist_data["portfolio_values"].append(portfolio_valuation)
        episode.hist_data["positions"].append(position)

        
    def on_episode_end(
        self,
        *,
        worker: RolloutWorker,
        
        base_env: BaseEnv,
        policies: Dict[str, Policy],
        episode: Episode,
        env_index: int,
        **kwargs,
    ):


        portfolio_valuation = episode.hist_data["portfolio_values"][-1]
        position_changes = np.sum(np.diff(episode.hist_data["positions"]) != 0)
        # m_port=np.mean(position_changes)
        
        episode.custom_metrics["portfolio_value"] = float(portfolio_valuation)
        episode.custom_metrics["position_changes"] = float(position_changes)
        # train.report(episode.custom_metrics)
        


In [40]:

ppo_algo = (PPOConfig().training(lr=ray.tune.loguniform(1e-8, 1e-5),
                         gamma=ray.tune.loguniform(.8, .999),      
                         lambda_=ray.tune.loguniform(.9, 1),
                         vf_loss_coeff=ray.tune.loguniform(.5, 1),     
                         kl_coeff=ray.tune.loguniform(.003,.3),
                         num_sgd_iter=ray.tune.randint(3,30),
                         clip_param=ray.tune.choice([.1,.2,.3]),
                         model={
                             'fcnet_hiddens': ray.tune.choice([[256,256],[512,512],[1024,1024],[2048,2048]]),
                            #  'use_attention':True,#tune.choice([True,False]),

                                },

                         )

                        .resources(
                                    num_gpus=0,
                                    
                                    )
                        .callbacks(TraderCallbacks)
                        .reporting(keep_per_episode_custom_metrics=False)
                        .environment(env="train_env")
                        .debugging(log_level='ERROR')
                            
                        .evaluation(
                                    evaluation_config={

                                                        "env": 'test_env',  # The dictionary we built before (only the overriding keys to use in evaluation)
                                                        "explore": False,
                                                        }
                                    )
                        
            
            )

ppo_algo_config=ppo_algo.to_dict()
# pprint(ppo_algo_config)

{'_AlgorithmConfig__prior_exploration_config': None,
 '_disable_action_flattening': False,
 '_disable_execution_plan_api': -1,
 '_disable_initialize_loss_from_dummy_batch': False,
 '_disable_preprocessor_api': False,
 '_dont_auto_sync_env_runner_states': False,
 '_enable_rl_module_api': -1,
 '_env_to_module_connector': None,
 '_evaluation_parallel_to_training_wo_thread': False,
 '_fake_gpus': False,
 '_is_atari': None,
 '_learner_class': None,
 '_learner_connector': None,
 '_model_config_dict': {},
 '_module_to_env_connector': None,
 '_per_module_overrides': {},
 '_rl_module_spec': None,
 '_run_training_always_in_thread': False,
 '_tf_policy_handles_more_than_one_loss': False,
 'action_mask_key': 'action_mask',
 'action_space': None,
 'actions_in_input_normalized': False,
 'add_default_connectors_to_env_to_module_pipeline': True,
 'add_default_connectors_to_learner_pipeline': True,
 'add_default_connectors_to_module_to_env_pipeline': True,
 'algorithm_config_overrides_per_module': {},


In [41]:
# ppo_algo.build()

In [42]:
# First, get the handle for the logger you want to modify
loggers=["ray.data","ray.tune","ray.rllib","ray.train","ray.serve","ray"]
for logger in loggers:
    ray_logger=logging.getLogger(logger)
    ray_logger .setLevel(logging.ERROR)

In [43]:
logger = logging.getLogger("ray")
logger.setLevel(logging.ERROR)
os.environ["RAY_IGNORE_UNHANDLED_ERRORS"]="1"

In [44]:
# ray.init(include_dashboard=False,
#         #  log_to_driver=False,
#          logging_level=logging.ERROR,
#          runtime_env = {
#                         "env_vars": {"PYTHONWARNINGS": "ignore"} }
#         #  _temp_dir='~/Users/sebastiancoombs/Documents/Git/MultiTrader/Rayrl_tune/tmp',
#         #  dashboard_host="0.0.0.0"
#          )

In [45]:


checkpoint_config=ray.train.CheckpointConfig(checkpoint_at_end=True,
                                         checkpoint_score_attribute='env_runners/custom_metrics/portfolio_value_mean',
                                        #  checkpoint_score_attribute='env_runners/episode_reward_mean',
                                        #  num_to_keep=1,
                                        #  checkpoint_frequency=10,
                                         )
reporter= JupyterNotebookReporter(
    sort_by_metric=True,
    metric='env_runners/custom_metrics/portfolio_value_max',
    mode='max',


    metric_columns={
                    'training_iteration':'iter',
                    'env_runners/reward':'reward',
                    'env_runners/custom_metrics/portfolio_value_max':'portfolio_value_max',
                    'env_runners/custom_metrics/position_changes_mean': 'position_changes_max',

                    'env_runners/episode_reward_mean':'episode_reward_mean',
                    'env_runners/custom_metrics/portfolio_value_mean': 'portfolio_value_mean',
                    'env_runners/custom_metrics/position_changes_mean': 'position_changes_mean',
                    'env_runners/custom_metrics/position_changes': 'position_changes',
                    
                    'env_runners/episode_reward_max': 'episode_reward_max',
                    'env_runners/episode_reward_min':'episode_reward_min',
                    'env_runners/custom_metrics/portfolio_value_min': 'portfolio_value_min',
                    

    },
    parameter_columns={

                        'lr':'lr',
                        'gamma':'gamma',
                        'lambda_':'lambda',
                        'kl_coeff':'kl_coeff',
                        'vf_loss_coeff':'vf_loss_coeff',
                        'num_sgd_iter':'num_sgd_iter',
                        'model/fcnet_hiddens':'fcnet_hiddens',
                        # 'model/use_attention':'use_attention'
                        }
    )

run_config=ray.train.RunConfig(

                            stop={
                                "training_iteration": 250,
                                #   "evaluation/episode_reward_mean": 1.5,
                                #   "timesteps_total":5000,
                                },
                                
                            
                            storage_path=checkpoint_path,
                            checkpoint_config=checkpoint_config,
                            progress_reporter=reporter

                        )

# search_algo = OptunaSearch(
#                             mode="max",                            
#                             metric='env_runners/custom_metrics/portfolio_value_mean',
# )



search_algo = HyperOptSearch(
    metric='env_runners/custom_metrics/portfolio_value_mean',
                            #  metric='episode_reward_mean',
                             mode='max',
                             )

search_algo= ConcurrencyLimiter(
                                search_algo, 
                                max_concurrent=1
                                )

tune_config=ray.tune.TuneConfig(num_samples=20,
                            reuse_actors=False,
                            search_alg=search_algo,
                            trial_name_creator=trial_str_creator,
                            trial_dirname_creator=trial_str_creator,

                            )
css = HTML('<link rel="stylesheet" type="text/css" href="./custom.css">') # make sure the path is correct, it fails silently otherwise
display(css)
tuner = ray.tune.Tuner(
    "PPO",
    param_space=ppo_algo_config,
    tune_config=tune_config,
    run_config=run_config,
    )
with ray.init(include_dashboard=False,
        #  log_to_driver=False,
        ignore_reinit_error=True,
        configure_logging =True,
         logging_level=logging.ERROR,
         log_to_driver=False,
         runtime_env ={"env_vars": {"PYTHONWARNINGS": "ignore::DeprecationWarning"}}

         ):
    appo_results=tuner.fit()

In [46]:


# appo_results=tuner.fit()

In [47]:
df_results = appo_results.get_dataframe()
df_results.to_csv('Agent_Tune/results/FX_appo_tune.csv')
# df_results

In [48]:

custom_cols=[
                'config/lr',
                'config/gamma',
                'config/lambda',
                'config/kl_coeff',
                'config/model/fcnet_hiddens',

                ]
custom_cols+=[c for c in df_results.filter(like='episode_reward').columns]
custom_cols+=[c for c in df_results.filter(like='custom_metrics').columns]
custom_cols=[c for c in custom_cols if 'sampler_results' not in c ]
custom_cols=[c for c in custom_cols if 'hist_stats' not in c ]
custom_cols=list(set(custom_cols))

In [49]:

df_results[custom_cols].sort_values('env_runners/custom_metrics/portfolio_value_mean',ascending=False)


,config/keep_per_episode_custom_metrics,config/model/fcnet_hiddens,config/gamma,config/lambda,env_runners/custom_metrics/portfolio_value_min,env_runners/episode_reward_min,env_runners/custom_metrics/position_changes_mean,config/lr,config/kl_coeff,env_runners/custom_metrics/portfolio_value_max,env_runners/episode_reward_mean,env_runners/custom_metrics/position_changes_min,env_runners/custom_metrics/position_changes_max,env_runners/custom_metrics/portfolio_value_mean,env_runners/episode_reward_max
5,False,"(1024, 1024)",0.900365,0.955820,1006.543483,24.761863,41.67,6.074078e-06,0.037562,1096.160639,81.012402,25.0,71.0,1045.973548,115.336049
11,False,"(512, 512)",0.840481,0.996872,1007.836869,-0.247703,46.18,5.687335e-06,0.146206,1093.315705,65.675585,24.0,90.0,1038.942366,109.664034
3,False,"(512, 512)",0.955536,0.949826,1009.665417,-2.029658,47.83,7.749627e-06,0.061098,1097.277296,65.852149,4.0,74.0,1038.597047,103.503430
13,False,"(1024, 1024)",0.856063,0.999940,979.723704,-39.799575,35.11,4.637492e-07,0.015720,1017.942981,7.346607,20.0,60.0,1002.842728,45.460048
1,False,"(2048, 2048)",0.836409,0.915298,980.188793,-32.016690,56.96,2.279680e-07,0.243094,1022.389198,1.808669,37.0,78.0,1001.616503,41.509079
15,False,"(256, 256)",0.962360,0.959553,975.317271,-35.573960,65.50,1.913576e-06,0.104614,1014.712586,6.598742,41.0,95.0,1001.257264,42.765950
19,False,"(256, 256)",0.883162,0.999055,979.903356,-31.595195,30.52,1.679901e-06,0.003336,1014.924381,8.032991,11.0,51.0,1001.109760,57.101896
0,False,"(512, 512)",0.802380,0.931155,981.528194,-40.269276,130.56,2.148616e-07,0.010039,1018.278266,2.407829,115.0,142.0,1000.966523,46.129425
10,False,"(512, 512)",0.879498,0.928609,971.074877,-35.863607,1.39,1.853007e-06,0.098265,1022.858563,2.549626,0.0,36.0,1000.746095,47.595714
7,False,"(1024, 1024)",0.870979,0.954526,977.011401,-55.254803,103.74,9.734042e-07,0.234293,1019.375103,0.876065,89.0,119.0,1000.745980,49.453396


In [50]:
# [c for c in df_results.columns]

In [51]:
best=appo_results.get_best_result(metric='env_runners/custom_metrics/portfolio_value_mean', mode='max')
best.checkpoint

Checkpoint(filesystem=local, path=/Users/sebastiancoombs/git/MultiTrader/Agent_FX/PPO_2024-06-30_11-49-05/PPO_ab3af569_trading_agent/checkpoint_000000)

In [52]:
agent= Algorithm.from_checkpoint(best.checkpoint.path)

(RolloutWorker pid=93056) Exception raised in creation task: The actor died because of an error raised in its creation task, ray::RolloutWorker.__init__() (pid=93056, ip=127.0.0.1, actor_id=9fb32ec331a2fac45d99f7d301000000, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x16fbcffd0>)
(RolloutWorker pid=93056)   File "/Users/sebastiancoombs/anaconda3/envs/agent_env/lib/python3.10/site-packages/gymnasium/envs/registration.py", line 740, in make
(RolloutWorker pid=93056)     env_spec = _find_spec(id)
(RolloutWorker pid=93056)   File "/Users/sebastiancoombs/anaconda3/envs/agent_env/lib/python3.10/site-packages/gymnasium/envs/registration.py", line 537, in _find_spec
(RolloutWorker pid=93056)     _check_version_exists(ns, name, version)
(RolloutWorker pid=93056)   File "/Users/sebastiancoombs/anaconda3/envs/agent_env/lib/python3.10/site-packages/gymnasium/envs/registration.py", line 403, in _check_version_exists
(RolloutWorker pid=93056)     _check_name_exists(ns, name)
(

EnvError: The env string you provided ('train_env') is:
a) Not a supported or -installed environment.
b) Not a tune-registered environment creator.
c) Not a valid env class string.

Try one of the following:
a) For Atari support: `pip install gym[atari] autorom[accept-rom-license]`.
   For PyBullet support: `pip install pybullet`.
b) To register your custom env, do `from ray import tune;
   tune.register('[name]', lambda cfg: [return env obj from here using cfg])`.
   Then in your config, do `config['env'] = [name]`.
c) Make sure you provide a fully qualified classpath, e.g.:
   `ray.rllib.examples.envs.classes.repeat_after_me_env.RepeatAfterMeEnv`


(raylet) [2024-07-01 00:54:23,688 E 93033 6853361] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-07-01_00-54-11_579159_91418 is over 95% full, available space: 901242880; capacity: 494384795648. Object creation will fail if spilling is required.
(raylet) [2024-07-01 00:54:33,762 E 93033 6853361] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-07-01_00-54-11_579159_91418 is over 95% full, available space: 900747264; capacity: 494384795648. Object creation will fail if spilling is required.
(raylet) [2024-07-01 00:54:43,829 E 93033 6853361] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-07-01_00-54-11_579159_91418 is over 95% full, available space: 899784704; capacity: 494384795648. Object creation will fail if spilling is required.
(raylet) [2024-07-01 00:54:53,885 E 93033 6853361] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-07-01_00-54-11_579159_91418 is over 95% full, available space: 899366912; capacity: 494384795648. Object creation

In [ ]:
pickle_cp_dir=f'FX_Agent/{symbol}_production_checkpoint'
prod_cp_dir=f'FX_Agent/{symbol}_production_agent'
prod_onnx_dir=f'FX_Agent/{symbol}_production_agent_onnx'

In [ ]:
num_epochs=500

bar=tqdm(range(num_epochs))
for i in bar:
    agent.train()
    if i % 100 == 0:
        checkpoint_dir = agent.save(pickle_cp_dir).checkpoint.path
        print(f"Checkpoint saved in directory {checkpoint_dir}")
        bar.set_description(f"Epoch {i}: Checkpoint saved in directory {checkpoint_dir}")

In [ ]:
test_env=NeuralForecastingTradingEnv(**test_env_config)

In [ ]:
obs,info=train_env.reset()
action=None
reward=0
terminated = truncated = False

while not terminated and not truncated:

    action = agent.compute_single_action(obs,explore=False)
    action = train_env.action_space.sample()
    # print(action)
    next_obs, reward, terminated, truncated, info = train_env.step(action)

    obs=next_obs
info
train_env.get_metrics(),info['portfolio_valuation']

In [ ]:
eval_profit=[]
for i in tqdm( range(100)):
    obs,info=train_env.reset()
    action=None
    reward=0
    terminated = truncated = False

    while not terminated and not truncated:

        action = agent.compute_single_action(obs,explore=False)
        # action = train_env.action_space.sample()
        # print(action)
        next_obs, reward, terminated, truncated, info = train_env.step(action)

        obs=next_obs

    eval_profit.append(info['portfolio_valuation'])
    info['portfolio_valuation']
profits=[e for e in eval_profit]
plt.plot(profits)
np.mean(profits)

In [ ]:
eval_profit=[]
for i in tqdm( range(100)):
    obs,info=test_env.reset()
    action=None
    reward=0
    terminated = truncated = False

    while not terminated and not truncated:

        action = agent.compute_single_action(obs,explore=False)
        # action = train_env.action_space.sample()
        # print(action)
        next_obs, reward, terminated, truncated, info = test_env.step(action)

        obs=next_obs

    eval_profit.append(info['portfolio_valuation'])
    info['portfolio_valuation']


In [ ]:

plt.plot(eval_profit)
np.mean(eval_profit)

In [ ]:
obs,info=test_env.reset()
action=None
reward=0
terminated = truncated = False

while not terminated and not truncated:

    action = agent.compute_single_action(obs,explore=False)
    # action = train_env.action_space.sample()
    # print(action)
    next_obs, reward, terminated, truncated, info = test_env.step(action)

    obs=next_obs


info['portfolio_valuation']

In [ ]:
policy1 = agent.get_policy(policy_id="default_policy")

# Tell RLlib to store an individual policy checkpoint (only for "pol1") inside
# /tmp/my_policy_checkpoint
policy1.export_checkpoint(prod_cp_dir)
policy1.export_model(prod_onnx_dir,onnx=18)